<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/paligemma/modelling_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
from torch import nn
from typing import Optional, Tuple, List
from torch.nn import CrossEntropyLoss
import math
import import_ipynb

ModuleNotFoundError: No module named 'siglip'